In [7]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from sklearn.preprocessing import StandardScaler

# Lettura dei file csv e selezione delle colonne di interesse
df1 = pd.read_csv('C:\\Users\\Utente\\Desktop\\cakio\\dcereijo-player-scores\\data\\uefarankteam3.csv')
df2 = pd.read_csv('C:\\Users\\Utente\\Desktop\\cakio\\dcereijo-player-scores\\data\\spi_global_rankings.csv')

# Esecuzione del matching tramite la funzione extractOne di fuzzywuzzy
df_merged = pd.merge(df2, df1, how='cross')
df_merged['correct_ratio'] = df_merged.apply(lambda x: fuzz.token_set_ratio(x['name22'], x['nome squadra']) if str(x['name22']) != 'nan' and str(x['nome squadra']) != 'nan' else 0, axis=1)
df_results = df_merged[df_merged['correct_ratio'] >= 95]

# Stampa dei risultati
print(df_results)

# Seleziona solo le righe relative ai paesi desiderati
desired_countries = ['Ger', 'Eng', 'Ita', 'Esp', 'Fra', 'Tur', 'Ned', 'por', 'Bel', 'Rus', 'Gre', 'Sco', 'Den']
df3 = df_results[df_results['paese'].isin(desired_countries)]

# Normalizza i punteggi di ogni variabile
scaler = StandardScaler()
df3[['off', 'def', 'spi', 'UEFAPoints']] = scaler.fit_transform(df3[['off', 'def', 'spi', 'UEFAPoints']])

# Assegna un peso diverso a ciascuna variabile
weights = {'off': 0.2, 'def': 0.2, 'spi': 0.2, 'UEFAPoints': 0.4}

# Calcola il punteggio overall per ogni squadra
df3['Overall'] = (df3['off']*weights['off'] + df3['def']*weights['def'] +
                 df3['spi']*weights['spi'] + df3['UEFAPoints']*weights['UEFAPoints'])

# Normalizza il punteggio overall in modo che vari da 0 a 1
df3['Overall'] = (df3['Overall'] - df3['Overall'].min()) / (df3['Overall'].max() - df3['Overall'].min())

# Stampa il dataframe risultante
print(df3)

# Raggruppa il dataframe per la colonna 'league' e calcola la somma degli overall
league_overall = df3.groupby('league')['Overall'].sum()

# Calcola la media degli overall per ogni lega
league_overall_mean = league_overall.mean()

# Ordina le leghe in base alla loro media di overall, dal più alto al più basso
sorted_leagues = league_overall.sort_values(ascending=False).index

# Crea un nuovo dataframe con la media di overall per ogni lega
new_df = pd.DataFrame({'league': sorted_leagues, 'mean_overall': league_overall.loc[sorted_leagues].values / len(df3['name22'].unique())})

# Stampa il nuovo dataframe
print(new_df)



        rank  prev_rank            name22                         league  \
4          1          1   Manchester City        Barclays Premier League   
568        2          2     Bayern Munich              German Bundesliga   
1134       3          4       Real Madrid       Spanish Primera Division   
1703       4          3         Barcelona       Spanish Primera Division   
2275       5          6         Liverpool        Barclays Premier League   
...      ...        ...               ...                            ...   
297924   526        528  Asteras Tripolis             Greek Super League   
303513   536        526       KV Oostende         Belgian Jupiler League   
304730   538        538          Giannina             Greek Super League   
305881   540        540   Shamrock Rovers  UEFA Europa Conference League   
334847   591        584    Brommapojkarna            Swedish Allsvenskan   

         off   def    spi  Unnamed: 0  posizione  nn  ... 17/18 18/19  19/20  \
4      

C:\Users\Utente\anaconda4\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
<ipython-input-7-a64d5b8ea05e>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Overall'] = (df3['off']*weights['off'] + df3['def']*weights['def'] +
<ipython-input-7-a64d5b8ea05e>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [9]:
# Normalizza il punteggio overall in modo che vari da 0 a 1
new_df['mean_overall'] = (new_df['mean_overall'] - new_df['mean_overall'].min()) / (new_df['mean_overall'].max() - new_df['mean_overall'].min())
# Stampa il nuovo dataframe
print(new_df)


                         league  mean_overall
0       Barclays Premier League      1.000000
1             German Bundesliga      0.958598
2      Spanish Primera Division      0.925981
3                 Italy Serie A      0.821485
4                French Ligue 1      0.533413
5              Dutch Eredivisie      0.513788
6        Belgian Jupiler League      0.381627
7    Turkish Turkcell Super Lig      0.336712
8          Russian Premier Liga      0.286198
9          Scottish Premiership      0.269551
10            Danish SAS-Ligaen      0.136140
11     Spanish Segunda Division      0.037737
12  English League Championship      0.036467
13   Argentina Primera Division      0.024147
14           Greek Super League      0.006290
15               French Ligue 2      0.000000
